In [ ]:
%pip install -U sagemaker boto3 --quiet
%pip install transformers datasets[s3] --quiet

In [5]:
from huggingface_hub.hf_api import HfFolder

# update the access token to download the tokenizer
access_token = "hf_insert-your-key-here"
HfFolder.save_token(access_token)

from transformers import AutoTokenizer

tokenizer_name = "meta-llama/Llama-2-70b-hf"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
block_size = 4096

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

c:\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aiman Hakimi\.cache\huggingface\hub\models--meta-llama--Llama-2-70b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# save training dataset to s3
training_input_path = f's3://{sess.default_bucket()}/neuronx_distributed/data'
print(f"uploading training dataset to: {training_input_path}")
train_dataset.save_to_disk(training_input_path)

print(f"uploaded data to: {training_input_path}")

In [ ]:
# no. of processes per node
PROCESSES_PER_NODE = 32
# no. of instances within the cluster, change this if you want to tweak the instance_count parameter
WORLD_SIZE = 32
# global batch size
GBS = 512
# input sequence length
SEQ_LEN = 4096
# pipeline parallel degree
PP_DEGREE = 8 # tensor parallel degree
TP_DEGREE = 8
# data paralell size
DP = ((PROCESSES_PER_NODE * WORLD_SIZE / TP_DEGREE / PP_DEGREE))
# batch size per model replica
BS = ((GBS / DP))
# number microbatches for pipeline execution. setting same as BS so each microbatch contains a single data sample
NUM_MICROBATCHES = BS
# no. of total steps for which to train model and adjusted to the step number when the loss function is approaching convergence
MAX_STEPS = 1500
# timeout in seconds for training. after this amount of time, sagemaker terminates the job regardless of its current status
MAX_RUN = 2 * (24 * 60 * 60)

In [ ]:
hyperparameters = {}
hyperparameters["train_batch_size"] = int(BS)
hyperparameters["use_meta_device_init"] = 1
# path where sagemaker uploads the training data
hyperparameters["training_dir"] = "/opt/ml/input/data/train" 
# config file containing llama 70b configuration , change this for tweaking the number of parameters
hyperparameters["training_config"] = "config.json" 

hyperparameters["max_steps"] = MAX_STEPS
hyperparameters["seq_len"] = SEQ_LEN
hyperparameters["pipeline_parallel_size"] = PP_DEGREE
hyperparameters["tensor_parallel_size"] = TP_DEGREE
hyperparameters["num_microbatches"] = int(NUM_MICROBATCHES)
hyperparameters["lr"] = 0.00015
hyperparameters["min_lr"] = 1e-05
hyperparameters["beta1"] = 0.9
hyperparameters["beta2"] = 0.95
hyperparameters["weight_decay"] = 0.1
hyperparameters["warmup_steps"] = 2000
hyperparameters["constant_steps"] = 0
hyperparameters["use_zero1_optimizer"] = 1
# the tensorboard logs will be stored here and eventually pushed to S3.
hyperparameters["tb_dir"] = "/opt/ml/checkpoints/tensorboard" 

In [ ]:
docker_image = f"763104351884.dkr.ecr.{region_name}.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.18.0-ubuntu20.04"

In [ ]:
import time

# training job name
job_name = f'llama-neuron-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
# checkpoint directory that contains the weights and other relevant data for the trained model
checkpoint_s3_uri = "s3://" + sagemaker_session_bucket + "/neuron_llama_experiment"
checkpoint_dir = '/opt/ml/checkpoints'</p><p>
In [ ]:
# neuron chache directory
cache_dir = "/opt/ml/checkpoints/neuron_cache"